In [58]:
import numpy as np
import torch 
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
#from torchsummary import summary
import matplotlib.pyplot as plt
import os
import time
import copy
from datetime import datetime
from collections import OrderedDict
import json
from torch.utils.tensorboard import SummaryWriter
import random
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split

In [59]:
data_dir = 'train_images'

time_run = datetime.now().strftime('%Y%m%d_%H%M%S')
save_dir = 'results/' + time_run
os.makedirs(save_dir, exist_ok=True)

transform_mean = transform_std = np.array([None, None, None])

In [60]:
df = pd.read_csv("train_labels.csv", names=["name", "label"], header=None)
df["image_path"] = df["name"].apply(lambda x: os.path.join(data_dir, x))
df = df[["image_path", "label"]]
df.head()

,image_path,label
0,train_images\train_0001.png,2012
1,train_images\train_0002.png,2003
2,train_images\train_0003.png,1994
3,train_images\train_0004.png,2014
4,train_images\train_0005.png,2003


In [61]:
df["image_path"][0]

'train_images\\train_0001.png'

In [62]:
n_epoch = 40
batch_size = 16
image_size = 256
input_size = 224
n_classes = 40

In [64]:
if transform_mean.any() == None:
    transform_mean = np.array([0.4914, 0.4822, 0.4465])
if transform_std.any() == None:
    transform_std = np.array([0.2470, 0.2435, 0.2616])


class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, df, data_dir, augment=True):
        
        if augment:
            transform = transforms.Compose([
                transforms.RandomCrop(128, padding=4),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean=transform_mean, std=transform_std)
            ])
        else:
            transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=transform_mean, std=transform_std)
            ])

        self.labels = df["label"]
        self.data_dir = data_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = df["image_path"][idx]
        image = Image.open(img_name).convert("RGB")   
        image = self.transform(image)
        
        label = df["label"][idx] - 1979
        sample = {'image': image, 'label': label}
        return sample


train_df, val_df = train_test_split(
                    df, test_size=0.2, random_state=42, shuffle=True, stratify=df["label"]
                    )    

train_dataset = ImageDataset(train_df, data_dir, augment=True)
val_dataset = ImageDataset(val_df, data_dir, augment=False)

train_df.to_csv("results/list_train.csv", header=None, index=None)
val_df.to_csv("results/list_val.csv", header=None, index=None)



train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True, 
                                           num_workers=0,
#                                           pin_memory=True,
                                           drop_last=True
                                          )

val_loader = torch.utils.data.DataLoader(val_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=False, 
                                           num_workers=0,
#                                           pin_memory=True,
                                           drop_last=False
                                         )

train_dataset_size = len(train_dataset)
val_dataset_size = len(val_dataset)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

In [65]:
## Model

class Resnet(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        resnet = models.resnet50(pretrained=True)
        self.resnet = nn.Sequential(*(list(resnet.children())[:-1]))
        for p in self.resnet.parameters():
            p.requires_grad = False
        self.dropout = nn.Dropout(p=0.5)
        self.fc = nn.Linear(2048, n_classes)
         
    def forward(self, x):
        x = self.resnet(x)
        x = F.adaptive_max_pool2d(x, 1)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        out = F.softmax(self.fc(x))
        return out

In [67]:
model = Resnet(n_classes)
model = model.to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
#optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4, nesterov=True)

# Decay LR by a factor of 0.1 every 7 epochs
#scheduler = lr_scheduler.StepLR(optimizer, step_size=9, gamma=0.33)

#T_max = n_epoch * train_dataset_size
#scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max, eta_min=0, last_epoch=-1)

global_step = 0

In [70]:
def train(epoch, model, loss_func, train_loader, optimizer, writer, scheduler=None):
    global global_step
    
    since = time.time()

    model.train()
    
    running_loss = 0.0            
    running_correct = 0

    for i, data in enumerate(train_loader):
        global_step +=1
        inputs, labels = data["image"], data["label"]
  
        scheduler.step()   
        writer.add_scalar('Train/LearningRate', scheduler.get_lr()[0], global_step)
        
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = loss_func(outputs, labels)
        _, predicted = torch.max(outputs.data, 1)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_correct += np.double(torch.sum(predicted == labels.data))
    train_loss = running_loss / train_dataset_size
    train_acc = running_correct / train_dataset_size

    elapsed = time.time() - since
    
    writer.add_scalar('Train/Loss', train_loss, epoch)
    writer.add_scalar('Train/Accuracy', train_acc, epoch)
    writer.add_scalar('Train/Time', elapsed, epoch)

    train_log = OrderedDict({
    'epoch':
    epoch,
    'train':
    OrderedDict({
        'loss': train_loss,
        'accuracy': train_acc,
        'time': elapsed,
        }),
    })
      
    print('Train Loss:{:.6f} Accuracy:{:.4f} Time:{:.1f}s'.format(train_loss, train_acc, elapsed))        
        
    return train_log, train_acc

def val(epoch, model, loss_func, val_loader, writer):
    global global_step
    since = time.time()
    
    model.eval()
    
    running_loss = 0.0            
    running_correct = 0
    
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            global_step +=1
            inputs, labels = data["image"], data["label"]
            
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = loss_func(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            print(predicted)

            running_loss += loss.item()
            running_correct += np.double(torch.sum(predicted == labels.data))
    val_loss = running_loss / val_dataset_size
    val_acc = running_correct / val_dataset_size

    elapsed = time.time() - since

    if epoch > 0:
        writer.add_scalar('Val/Loss', val_loss, epoch)
        writer.add_scalar('Val/Accuracy', val_acc, epoch)
        writer.add_scalar('Val/Time', elapsed, epoch)    
    
    val_log = OrderedDict({
    'epoch':
    epoch,
    'val':
    OrderedDict({
        'loss': val_loss,
        'accuracy':val_acc,
        'time': elapsed,
        }),
    })
    
    print('Val Loss:{:.6f} Accuracy:{:.4f} Time:{:.1f}s'.format(val_loss, val_acc, elapsed))  
    
    return val_log, val_acc


def main():
    print(time_run)
    train_since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

######################## This part function not verified ###################    
    # set random seed
    seed = 17 # arbitrary
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
###########################################################################    
    writer = SummaryWriter(save_dir)
    
    # run test before start training? whats that??????????
    val(0, model, loss_func, val_loader, writer)
    print()
    epoch_logs = []
    train_accs = []
    val_accs = []
    
    for epoch in range(n_epoch):
        print('Epoch {}/{}'.format(epoch, n_epoch - 1))
        print('-' * 10)
        
        train_log, train_acc = train(epoch, model, loss_func, train_loader, optimizer, writer, scheduler)
        val_log, val_acc = val(epoch, model, loss_func, val_loader, writer)
        
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
        

        print()
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        epoch_log = train_log.copy()
        epoch_log.update(val_log)
        epoch_logs.append(epoch_log)
        with open(save_dir+'/{}_log.json'.format(time_run), 'w') as fout:
            json.dump(epoch_logs, fout, indent=2)
        
        if epoch % 50 == 0 and epoch !=0:
            torch.save(model.state_dict(), save_dir+'/{}_ep{}_acc{:.4f}.pth'.format(time_run, epoch, val_acc))
    
    accuracies = {'train_accuracy':np.array(train_accs), 'val_accuracy':np.array(val_accs)}
    np.save(save_dir+'/{}_accs.npy'.format(time_run), accuracies)
    
    train_elapsed = time.time() - train_since
    print('Training complete in {:.0f}m {:.0f}s'.format(train_elapsed // 60, train_elapsed % 60))
    print('Best val Acc: {:.4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), save_dir+'/{}_bestacc{:.4f}.pth'.format(time_run, best_acc))

In [71]:
if __name__ == '__main__':
    main()

20191229_001436


C:\Users\silas\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([25, 19, 16, 33, 13, 33, 33, 13, 17, 14, 33, 16, 33, 14, 25, 19],
       device='cuda:0')
tensor([33, 25, 17, 25, 17, 13, 13, 33, 14, 14, 33, 25, 19, 13, 25, 19],
       device='cuda:0')
tensor([14, 33, 14, 14, 33, 13, 16, 19, 13, 25, 14, 33, 25, 33, 13, 16],
       device='cuda:0')
tensor([13, 16, 25, 13, 14, 13, 14, 25, 33, 17, 17, 14, 17, 16, 13, 14],
       device='cuda:0')
tensor([19, 33, 16, 25, 16, 25, 25, 17, 17, 18, 25, 13, 13, 13, 14, 13],
       device='cuda:0')
tensor([33, 33, 25, 14, 14, 19, 13, 17, 17, 33, 13, 13, 13, 25, 25, 14],
       device='cuda:0')
tensor([14, 25, 25, 25, 33, 33, 25, 25, 16, 17, 14, 17, 25, 25, 25, 25],
       device='cuda:0')
tensor([13, 19, 14, 16, 19, 25, 16, 14, 13, 25, 16, 25, 14, 16, 14, 25],
       device='cuda:0')
tensor([13, 16, 13, 14, 13, 14, 19, 25, 17, 17, 25, 17, 13, 19, 13, 25],
       device='cuda:0')
tensor([17, 19, 19, 25, 25, 16, 17, 13, 25, 17, 25, 17, 13, 17, 16, 25],
       device='cuda:0')
tensor([25, 25, 14, 13, 17, 16

KeyboardInterrupt: 

In [76]:
test_model = Resnet(n_classes)
test_model.load_state_dict(torch.load("results/20191229_001436/20191229_001436_bestacc0.1861.pth"))
test_model.to(device)

test_dir = "test_images"

test_df = pd.read_csv("sample_submission.csv", names=["name", "label"], header=None)
test_df["image_path"] = test_df["name"].apply(lambda x: os.path.join(test_dir, x))

test_dataset = ImageDataset(test_df, test_dir, augment=False)

test_loader = torch.utils.data.DataLoader(test_dataset, 
                                           batch_size=1, 
                                           shuffle=False, 
                                           num_workers=0,
#                                           pin_memory=True,
                                           drop_last=False
                                         )

for i, data in enumerate(test_loader):
    inputs, labels = data["image"], data["label"]
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = test_model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    predicted = int(predicted) + 1979
    test_df["label"][i] = predicted

test_df[["name", "label"]].to_csv("submission.csv", header=None, index=False)

Resnet(
  (resnet): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
          (0): Conv2d(64, 256, kernel_

C:\Users\silas\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


2004


C:\Users\silas\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1992
2004
1992
2004
2004
1992
2004
1992
1998
1992
2004
1992
1992
1995
1992
1995
2004
1992
2004
1992
1992
2004
1995
1992
2004
2004
2004
2004
1992
2004
2004
2004
1995
1992
1992
1995
1992
2004
2004
1992
1992
2004
2004
1992
1995
2004
2004
1992
2004
2004
2004
1992
2004
1992
1992
1992
2004
1992
1992
2004
2004
2004
1992
1992
1998
1992
1995
2004
1992
1992
1992
1992
1992
1992
1992
1992
1992
2004
2004
1995
1992
1992
1992
1992
1992
1995
1992
2004
1992
2004
1998
1992
1998
1992
1992
1992
1992
2004
1992
1992
2004
2004
1998
2004
2004
1995
1992
1992
1992
1992
2004
1992
2004
1992
2004
1992
2004
1992
1992
1992
2004
1992
1992
1992
2004
2004
1995
1992
1995
1992
2004
1992
2004
1992
2004
2004
1992
1992
1992
1998
2004
2004
1995
1995
1992
1992
1995
1992
1995
2004
1992
1992
2004
1992
1992
1992
1992
1992
1992
1992
1992
2004
1992
1992
1992
2004
1992
1995
2004
1992
1992
1992
2004
1992
1992
2004
2004
1992
1992
2004
1998
2004
1992
1992
1995
2004
2004
2004
2004
2004
2004
1992
1992
1995
2004
1992
1992
1992
1992
2004


In [ ]:
test_df[["name", "label"]].head()